**GET FINE-TUNED MODEL FROM HUGGINGFACE**


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("cfstar188/csc392-model-2-final")
model = AutoModelForCausalLM.from_pretrained("cfstar188/csc392-model-2-final")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
!pip3 install torch

In [ ]:
import torch

device = torch.device("cuda")
model.cuda()

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), eps

**FUNCTIONS FOR INFERENCING**

In [ ]:
import time

def get_completion(annotations: str, model, tokenizer) -> str:
    device = "cuda:0"

    # Start measuring time
    start_time = time.time()

    # Format the prompt
    prompt_template = """"
        <start_of_turn>user
        Based on the following annotated Asciinema recording:
        {annotations}
        Generate a single cohesive Markdown file that explains and summarizes the entire process from the Asciinema recording.
        The summary should focus on:
        * Describing the context of each action.
        * Clearly explaining why the action was used, what the action does, and the outcome of this action.
        * Focus on making the documentation informative enough to be easily followed, capturing the step-by-step workflow as demonstrated in the recording, without using generic phrases.

        In the Markdown file, I want the following:
        * The main title should be preceded with a # and section titles should be preceded with ##.
        * End the markdown file with a 'Key Takeaways' section that summarizes the main points of the recording.
        <end_of_turn>\n
        <start_of_turn>model
    """
    prompt = prompt_template.format(annotations=annotations)

    # Tokenize the prompt
    encodings = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    # Move inputs to the correct device
    model_inputs = encodings.to(device)

    # Generate the model's response
    generated_ids = model.generate(**model_inputs, max_new_tokens=int(len(annotations)//1.5), do_sample=True, pad_token_id=tokenizer.eos_token_id)

    # Decode the output tokens
    decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # End timing
    end_time = time.time()

    # Print time
    print(f"Total time: {end_time - start_time:.2f} seconds")

    # Print and return the result
    return decoded

In [ ]:
import re
import json

def parse_output(input_string):
    # Find the position of the word "model"
    index_model = input_string.find("model")

    # If "model" is found, get the substring after it

    if index_model == -1:
      return "model keyword not found"
    else:
        input_string = input_string[index_model+5:] # get all characters after the word "model"

    # Find the position of the substring "Key Takeaways"
    index_takeaways = input_string.find("Key Takeaways")

    # If "Key Takeaways" is found, extract it and the next 5 lines
    if index_takeaways != -1:
        # Extract the substring starting from "Key Takeaways"
        takeaways_substring = input_string[index_takeaways:]

        # Split into lines and get the first lines after Key Takeaways
        lines = takeaways_substring.splitlines()

        # Find the index of the first blank line
        blank_line_index = 0
        for index, line in enumerate(lines):
            if line == '':
                blank_line_index = index
                break

        lines = lines[:blank_line_index]

        # Join the lines back into a string
        return (input_string[:index_takeaways] + "\n".join(lines)).strip()
    else: # if Ket Takeaways is not found, just return the whole string
        return input_string

def save_markdown(file_path, content):
    with open(file_path, 'w') as file:
        file.write(content)

"""
Return an array of all <data-piece>...</data-piece> in the input string
"""
def extract_data_pieces(input_string):
    # Define the regex pattern to match <data-piece>...</data-piece>
    pattern = r"<data-piece>.*?</data-piece>"
    # Use re.findall to extract all matches
    data_pieces = re.findall(pattern, input_string, re.DOTALL)
    return data_pieces

"""
Precondition: file_path is a json file with an "annotation" key that stores an annotated session
Postcondition: Generates the markdown summary file in the same directory as the json file
"""
def generate_markdown(file_path):
  json_content = None
  with open(file_path, 'r') as f:
    json_content = json.load(f)
    # Check if json_content is a dictionary and extract the "annotation" key
    if isinstance(json_content, dict):
        json_content = json_content.get("annotation")

        json_content = extract_data_pieces(json_content)
        parsed_output = ""
        for content in json_content:
          completion = get_completion(content, model=model, tokenizer=tokenizer)
          parsed_output += parse_output(completion) + "\n\n"

        print(parsed_output)
        inference_file_name = file_path.replace(".json", "_inference.md")
        save_markdown(inference_file_name, parsed_output)

**TESTS**

In [ ]:
generate_markdown("49/data.json")

Total time: 38.94 seconds
# Asciinema Recording Documentation: SSH and Install Asciinema on Server

## Overview
This recording demonstrates the process of SSHing into a server, installing `asciinema`, and then logging out of the server.

---

## Timeline Breakdown

### 1. **SSH Into Server**
- **Command:** `ssh 10.0.7.138`
- **Output:** `demo@10.0.7.138's password:`
- **Explanation:** The user initiates an SSH connection to the server at IP address `10.0.7.138`.

### 2. **Entering Password for SSH Login**
- **Command:** `1M3T567!`
- **Output:** 
    ```bash
    Linux boxtop 6.6.13-amd64 1 SMP PREEMPT_DYNAMIC Debian 6.6.13-1 (2024-01-20) x86_64
    Plan your installation, and FAI installs your plan.
    Last login: Sun Sep 22 12:2417 2024
    demo@boxtop:~$
    ```
- **Explanation:** The user enters the password to successfully log into the server.

### 3. **Installing Asciinema**
- **Command:** `sudo apt install asciinema`
- **Output:** `[sudo] password for demo:`
- **Explanation:** Th

In [ ]:
generate_markdown("48/data.json")

Total time: 150.41 seconds
# Asciinema Recording Documentation: Cloning and Setting Up deerhacks-cms

## Overview
This recording demonstrates the cloning and setup process for the deerhacks-cms repository within the MCSS directory.

---

## Timeline Breakdown

### 1. **Cloning the Repository**
- **Command:** `git clone https://github.com/utmmcss/deerhacks-cms.git`
- **Output:**
    ```bash
    Cloning into 'deerhacks-cms'...
    remote: Enumerating objects: 230, done.
    remote: Counting objects: 0% (1/230)
    remote: Counting objects: 1% (3/230)
    remote: Counting objects: 2% (5/230)
    remote: Counting objects: 3% (7/230)
    remote: Counting objects: 4% (10/230)
    remote: Counting objects: 5% (12/230)
    remote: Counting objects: 6% (14/230)
    remote: Counting objects: 7% (17/230)
    remote: Counting objects: 8% (19/230)
    remote: Counting objects: 9% (21/230)
    remote: Counting objects: 10% (23/230)
    remote: Counting objects: 11%
    ```
- **Explanation:** The use

In [ ]:
generate_markdown("45/data.json")

Total time: 44.25 seconds
# Asciinema Recording Documentation: Managing Files in user's Home Directory

## Overview
This recording demonstrates how to perform file management tasks in the `/home/user` directory.

---

## Timeline Breakdown

### 1. **Searching for Files**
- **Command:** `find /home/user -name '*.txt'`
- **Output:** `/home/user/documents/file1.txt`
- **Explanation:** The user searches for `.txt` files in the `/home/user` directory.

### 2. **Searching for Directories**
- **Command:** `find /home/user -type d`
- **Output:** `/home/user/documents /home/user/images`
- **Explanation:** The user searches for directories inside `/home/user`, including both the `documents` and `images` directories.

### 3. **Ending the Session**
- **Command:** `exit`
- **Output:** *(none)*
- **Explanation:** The user exits the SSH session after completing the file management tasks.

---

## Key Takeaways
- **File Searching:** Demonstrated using `find` to locate specific files.
- **Directory Man

In [ ]:
generate_markdown("34/data.json")

Total time: 36.77 seconds
# Asciinema Recording Documentation: File Copy and Management

## Overview
This recording demonstrates copying files, renaming files, and deleting files in the Linux terminal.

---

## Timeline Breakdown

### 1. **Copying a File**
- **Command:** `cp file1.txt file2.txt`
- **Output:** *(none)*
- **Explanation:** The user copies the contents of `file1.txt` to a new file `file2.txt`.

### 2. **Verifying the Copy**
- **Command:** `ls`
- **Output:** `file1.txt file2.txt`
- **Explanation:** The user lists the directory contents to confirm that both `file1.txt` and `file2.txt` exist.

### 3. **Renaming the File**
- **Command:** `mv file2.txt file3.txt`
- **Output:** *(none)*
- **Explanation:** The user renames `file2.txt` to `file3.txt`.

### 4. **Deleting the Renamed File**
- **Command:** `rm file3.txt`
- **Output:** *(none)*
- **Explanation:** The user deletes the file `file3.txt` after renaming it.

### 5. **Ending the Session**
- **Command:** `exit`
- **Output:**

In [ ]:
generate_markdown("46/data.json")

Total time: 123.84 seconds
# Asciinema Recording Documentation: Running the Local Development Server for deerhacks

## Overview
This recording demonstrates running the local development server for the deerhacks project using `yarn`.

---

## Timeline Breakdown

### 1. **Navigating to Project Directory**
- **Command:** `cd deerhacks/`
- **Output:** *(none)*
- **Explanation:** The user navigates into the `deerhacks` project directory.

### 2. **Running the Local Development Server**
- **Command:** `yarn dev`
- **Output:** `warning ../../package.json: No license field [PWA] PWA support is disabled local: http://localhost:3000`
- **Explanation:** The user runs the development server using `yarn` to serve the web app locally on `localhost:3000`.

---

## Key Takeaways
- **Local Development Setup:** Demonstrated how to start the development server for the deerhacks project using `yarn`.
- **PWA Support:** Showcased that PWA support is disabled by default in Next.js.


